# Yelp data

You can use the tool below to pull in search results from Yelp for businesses and other establishments in a given neighborhood. The tool is currently set up to pull in all search results for Red Hook regardless of the establishment type. The API call will return lists containing dictionaries. Each dictionary corresponds to one specific establishment.

Note that in order to run the code, you will need to enter your own API key in the following cell. You can also refine the searches by adding specific search terms (like "bar" or "restaurant"), price, and other search tools available on Yelp.

In [1]:
# Libraries
import requests
import json
import pandas as pd
import geopandas as gpd
import shapely
from fiona.crs import from_epsg
import matplotlib.pylab as plt
import urllib.request
import copy

try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

In [2]:
API_KEY = open('yelp_api.txt', 'r').readlines()[0][:].rstrip() # api stored in seperate .txt file

## Functions for API calls

In [3]:
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
# API_KEY = open('yelp_api.txt', 'r').readlines()[0][:] # api stored in seperate .txt file

# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
REVIEW_PATH = '/v3/businesses/{}/reviews'

# Set search limit due to API restrictions
SEARCH_LIMIT = 50

def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

def radius_search(api_key, latitude, longitude, radius, offset):
    """Query the Search API
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'latitude': latitude,
        'longitude': longitude,
        'radius': radius,
        'limit': SEARCH_LIMIT,
        'offset': offset
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

def query_api(latitude, longitude, radius, offset):
    # Queries the API by the input values from the user.
    response = radius_search(API_KEY, latitude, longitude, radius, offset)

    businesses = response.get('businesses')
    return businesses

def get_review(api_key, biz_id):
    return request(API_HOST, REVIEW_PATH.format(biz_id), api_key)

# Still working on this part, not sure how to get it to work right (but the previous function works)
def get_reviews(api_key, biz_ids):
    responses = []
    for biz_id in biz_ids:
        response = request(API_HOST, REVIEW_PATH.format(biz_id), api_key)
        responses.append(response)
    return responses

## Getting establishments near site

In [4]:
# Create offsets for multiple API calls since Yelp limits the number of requests one can make at a time
offsets = [0]
# Control the total number of results using the second argument in the below range (50 * __)
for i in range(1,10):
    off = i*50
    offsets.append(off)

In [5]:
# Set search radius (in meters)
radius = 400

In [6]:
"""
Loop through multiple queries to get the first 500 results for Red Hook.

Note that the API call results in 500 dictionaries - each dictionary contains the information for one business.
"""
results = []
for i in range(len(offsets)):
    # Using the longitude and latitude of the site centroid
    businesses = query_api(40.67840802364635, -74.01521633676086, radius, offsets[i])
    results.append(businesses)

In [7]:
# To see how many total results were received from query
for i in range(len(results)):
    print("Batch {}: {} results".format(i,len(results[i])))

Batch 0: 18 results
Batch 1: 0 results
Batch 2: 0 results
Batch 3: 0 results
Batch 4: 0 results
Batch 5: 0 results
Batch 6: 0 results
Batch 7: 0 results
Batch 8: 0 results
Batch 9: 0 results


In [8]:
# Sample of result from API call - this particular dictionary includes data on Hometown BBQ
results[0][0]

{'id': 'Ms3CAGddVbgetiQrpzqxPQ',
 'alias': 'hometown-bar-b-que-brooklyn-3',
 'name': 'Hometown Bar-B-Que',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Eo9NJvaF8j9HLa0GX9yNUA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/hometown-bar-b-que-brooklyn-3?adjust_creative=blQ-cNUMXpZs8T2qda_yow&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=blQ-cNUMXpZs8T2qda_yow',
 'review_count': 1192,
 'categories': [{'alias': 'bbq', 'title': 'Barbeque'},
  {'alias': 'smokehouse', 'title': 'Smokehouse'}],
 'rating': 4.0,
 'coordinates': {'latitude': 40.6748965703426, 'longitude': -74.0160489746129},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '454 Van Brunt St',
  'address2': '',
  'address3': None,
  'city': 'Brooklyn',
  'zip_code': '11231',
  'country': 'US',
  'state': 'NY',
  'display_address': ['454 Van Brunt St', 'Brooklyn, NY 11231']},
 'phone': '+13472944644',
 'display_phone': '(347) 294-4644',
 'distance': 396.7187161958546}

In [10]:
# Make a df from the results
biz_df = pd.DataFrame(results[0])
biz_df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,hometown-bar-b-que-brooklyn-3,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...","{'latitude': 40.6748965703426, 'longitude': -7...",(347) 294-4644,396.718716,Ms3CAGddVbgetiQrpzqxPQ,https://s3-media2.fl.yelpcdn.com/bphoto/Eo9NJv...,False,"{'address1': '454 Van Brunt St', 'address2': '...",Hometown Bar-B-Que,+13472944644,$$,4.0,1192,[],https://www.yelp.com/biz/hometown-bar-b-que-br...
1,steves-authentic-key-lime-pies-brooklyn,"[{'alias': 'bakeries', 'title': 'Bakeries'}]","{'latitude': 40.6777599765016, 'longitude': -7...",(718) 858-5333,250.920284,CoJmKwsXt3esEKVB6-3EkQ,https://s3-media1.fl.yelpcdn.com/bphoto/O-xxrP...,False,"{'address1': '185 Van Dyke St', 'address2': ''...",Steve's Authentic Key Lime Pies,+17188585333,$$,4.5,383,[],https://www.yelp.com/biz/steves-authentic-key-...
2,baked-brooklyn,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...","{'latitude': 40.676789, 'longitude': -74.013211}",(718) 222-0345,246.996392,Q_7J5E-cYCQfHNCkCyMdLA,https://s3-media1.fl.yelpcdn.com/bphoto/vpA4Pd...,False,"{'address1': '359 Van Brunt St', 'address2': '...",Baked,+17182220345,$$,4.0,495,[],https://www.yelp.com/biz/baked-brooklyn?adjust...
3,the-good-fork-brooklyn,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.67599, 'longitude': -74.01432}",(718) 643-6636,284.176105,-BOAKHyKKAXE1WhuVpMFkQ,https://s3-media2.fl.yelpcdn.com/bphoto/JFWO8Z...,False,"{'address1': '391 Van Brunt St', 'address2': '...",The Good Fork,+17186436636,$$,4.0,397,[],https://www.yelp.com/biz/the-good-fork-brookly...
4,fort-defiance-brooklyn,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 40.6766128540039, 'longitude': -7...",(347) 453-6672,244.282640,lQKKBtdp7No9HJ_tW6PuAQ,https://s3-media1.fl.yelpcdn.com/bphoto/un9aIE...,False,"{'address1': '365 Van Brunt St', 'address2': '...",Fort Defiance,+13474536672,$$,4.0,331,[],https://www.yelp.com/biz/fort-defiance-brookly...


In [11]:
print("Number of establishments listed on Yelp within {} meters of site centroid: {}".format(radius, \
                                                                                             biz_df.shape[0]))

Number of establishments listed on Yelp within 400 meters of site centroid: 18


## Getting Yelp reviews from nearby establishments

Here's more info on the API query for Yelp reviews:

https://www.yelp.com/developers/documentation/v3/business_reviews

Note that this API call is limited to only 3 reviews per establishment.

In [12]:
# Use the businesses that were returned from the previous query
biz_ids = list(biz_df.id)

In [13]:
# API call for reviews
reviews = []
for biz_id in biz_ids:
    review = get_review(API_KEY, biz_id)
    reviews.append(review)

In [14]:
# Make a df from those reviews
reviews_df = pd.DataFrame(reviews)
reviews_df.head()

,possible_languages,reviews,total
0,"[fr, en, it, ja]","[{'id': 'gfJfccAseabkH9Hhp41Qww', 'url': 'http...",1192
1,"[fr, en, it]","[{'id': '-Pcn4OpK7B9opjbOIVv44g', 'url': 'http...",383
2,[en],"[{'id': '63UNNOriaxChZdtugnBhuw', 'url': 'http...",495
3,[en],"[{'id': 'S7lv-5qAj9bAvXxrekpHzA', 'url': 'http...",397
4,"[fr, en, sv]","[{'id': 'gomGKFtb8mZAR2XCF8dSzw', 'url': 'http...",331


In [15]:
# Split the reviews into separate columns
reviews_df = pd.concat([reviews_df, reviews_df['reviews'].apply(pd.Series)], axis = 1).drop('reviews', \
                                                                                               axis = 1)
# And break out each part of each review into separate columns
for i in range(3):
    df_sep = reviews_df[i].apply(pd.Series)
    df_sep.rename(columns={'id':'id_{}'.format(i),'url':'url_{}'.format(i),'text':'text_{}'.format(i),\
                           'rating':'rating_{}'.format(i),'time_created':'time_created_{}'.format(i),\
                           'user':'user_{}'.format(i)}, inplace=True)
    reviews_df = pd.concat([reviews_df, df_sep], axis = 1).drop([i], axis = 1)

reviews_df.head()

,possible_languages,total,id_0,url_0,text_0,rating_0,time_created_0,user_0,id_1,url_1,text_1,rating_1,time_created_1,user_1,id_2,url_2,text_2,rating_2,time_created_2,user_2
0,"[fr, en, it, ja]",1192,gfJfccAseabkH9Hhp41Qww,https://www.yelp.com/biz/hometown-bar-b-que-br...,What a solid BBQ joint! \n\nMy friends and I c...,5,2019-04-20 13:47:44,"{'id': 'RoO8V10M8wLrJT-JnuNVig', 'profile_url'...",h066kAT2eX89It6WPSJYPw,https://www.yelp.com/biz/hometown-bar-b-que-br...,Incredible BBQ. Loved every bite of the Briske...,4,2019-04-13 19:01:43,"{'id': 'gthIh2LBOUDwdjI4atXS6A', 'profile_url'...",hrs40idQeROAxKFEFqhFkQ,https://www.yelp.com/biz/hometown-bar-b-que-br...,This is a solid bbq joint. I would go back. ...,3,2019-04-03 07:48:35,"{'id': 'gFptgYG33B3aZqs_se7zsg', 'profile_url'..."
1,"[fr, en, it]",383,-Pcn4OpK7B9opjbOIVv44g,https://www.yelp.com/biz/steves-authentic-key-...,These pies are better than any Key Lime pie I'...,5,2019-04-17 15:33:09,"{'id': 'ITvPLa0RztVl-oxbattSXg', 'profile_url'...",E0aYVZTEQOmFMKdnlH6Ayg,https://www.yelp.com/biz/steves-authentic-key-...,"The key lime pie was very good, though maybe n...",5,2018-09-06 10:11:44,"{'id': 'yh8_9Ix1wp9dekvyOZbilQ', 'profile_url'...",bMLNPGFWQnSSzS8EDX0T0w,https://www.yelp.com/biz/steves-authentic-key-...,Everything you've heard of is true and then so...,4,2019-03-09 19:39:00,"{'id': 'wwr-V0x5Li7uSywDcNuvWg', 'profile_url'..."
2,[en],495,63UNNOriaxChZdtugnBhuw,https://www.yelp.com/biz/baked-brooklyn?adjust...,"I'm drooling writing this review. Great, now I...",5,2019-03-27 16:55:56,"{'id': 'bt51F2SgYVcPWvNuDwZwXQ', 'profile_url'...",eo5zpZ5ene8-humhz6vkBw,https://www.yelp.com/biz/baked-brooklyn?adjust...,Love Baked!! It is a block from my daughter's ...,4,2019-04-20 11:40:09,"{'id': 'W8AK0XlpePkv2epHeqLWxg', 'profile_url'...",aNFenmhOPRMwykJj7Zgdnw,https://www.yelp.com/biz/baked-brooklyn?adjust...,Coffee/pastries are great but literally every ...,1,2019-04-19 10:40:57,"{'id': 'hotAE5Qt-0CubkmHzzJ7Ng', 'profile_url'..."
3,[en],397,S7lv-5qAj9bAvXxrekpHzA,https://www.yelp.com/biz/the-good-fork-brookly...,One of my favorite itineraries for showing out...,5,2018-11-21 07:52:57,"{'id': '-yEhhXT6URxh_yxHko5Gzg', 'profile_url'...",FuZ4iXW-hq5oK-U6rNm0SQ,https://www.yelp.com/biz/the-good-fork-brookly...,"I went in August and it was great, but most re...",2,2019-04-04 10:10:39,"{'id': 'SwvoOnfwvtxVDZOX_JcRGA', 'profile_url'...",iTe5wvWRMrG5e3WLzgvr7A,https://www.yelp.com/biz/the-good-fork-brookly...,Was our first time to The Good Fork today - wa...,2,2019-04-21 18:57:01,"{'id': 'XKhm98gVInrRxOKPYkp9vw', 'profile_url'..."
4,"[fr, en, sv]",331,gomGKFtb8mZAR2XCF8dSzw,https://www.yelp.com/biz/fort-defiance-brookly...,"I came here to try ""the best Irish Coffee in t...",5,2018-12-20 18:00:52,"{'id': 'u_xu22T8i9PW7UcHPR6k9Q', 'profile_url'...",tyJfcZuhlkf7vO8M8zMQoQ,https://www.yelp.com/biz/fort-defiance-brookly...,This is a cute little spot in Red Hook that is...,4,2018-10-12 14:03:06,"{'id': 'eAw1TQ-FCrPmlgYYkG-ahg', 'profile_url'...",K5iH6pnSn5DtEGnCzReRow,https://www.yelp.com/biz/fort-defiance-brookly...,I'm going to say it- everybody who is reviewin...,5,2019-04-23 05:33:44,"{'id': 'b2RrDYWKkwBtMrGpgpoANQ', 'profile_url'..."


In [16]:
# Merge back with the establishment names
reviews_df = pd.concat([reviews_df,biz_df.name],axis=1)
reviews_df.head()

,possible_languages,total,id_0,url_0,text_0,rating_0,time_created_0,user_0,id_1,url_1,...,rating_1,time_created_1,user_1,id_2,url_2,text_2,rating_2,time_created_2,user_2,name
0,"[fr, en, it, ja]",1192,gfJfccAseabkH9Hhp41Qww,https://www.yelp.com/biz/hometown-bar-b-que-br...,What a solid BBQ joint! \n\nMy friends and I c...,5,2019-04-20 13:47:44,"{'id': 'RoO8V10M8wLrJT-JnuNVig', 'profile_url'...",h066kAT2eX89It6WPSJYPw,https://www.yelp.com/biz/hometown-bar-b-que-br...,...,4,2019-04-13 19:01:43,"{'id': 'gthIh2LBOUDwdjI4atXS6A', 'profile_url'...",hrs40idQeROAxKFEFqhFkQ,https://www.yelp.com/biz/hometown-bar-b-que-br...,This is a solid bbq joint. I would go back. ...,3,2019-04-03 07:48:35,"{'id': 'gFptgYG33B3aZqs_se7zsg', 'profile_url'...",Hometown Bar-B-Que
1,"[fr, en, it]",383,-Pcn4OpK7B9opjbOIVv44g,https://www.yelp.com/biz/steves-authentic-key-...,These pies are better than any Key Lime pie I'...,5,2019-04-17 15:33:09,"{'id': 'ITvPLa0RztVl-oxbattSXg', 'profile_url'...",E0aYVZTEQOmFMKdnlH6Ayg,https://www.yelp.com/biz/steves-authentic-key-...,...,5,2018-09-06 10:11:44,"{'id': 'yh8_9Ix1wp9dekvyOZbilQ', 'profile_url'...",bMLNPGFWQnSSzS8EDX0T0w,https://www.yelp.com/biz/steves-authentic-key-...,Everything you've heard of is true and then so...,4,2019-03-09 19:39:00,"{'id': 'wwr-V0x5Li7uSywDcNuvWg', 'profile_url'...",Steve's Authentic Key Lime Pies
2,[en],495,63UNNOriaxChZdtugnBhuw,https://www.yelp.com/biz/baked-brooklyn?adjust...,"I'm drooling writing this review. Great, now I...",5,2019-03-27 16:55:56,"{'id': 'bt51F2SgYVcPWvNuDwZwXQ', 'profile_url'...",eo5zpZ5ene8-humhz6vkBw,https://www.yelp.com/biz/baked-brooklyn?adjust...,...,4,2019-04-20 11:40:09,"{'id': 'W8AK0XlpePkv2epHeqLWxg', 'profile_url'...",aNFenmhOPRMwykJj7Zgdnw,https://www.yelp.com/biz/baked-brooklyn?adjust...,Coffee/pastries are great but literally every ...,1,2019-04-19 10:40:57,"{'id': 'hotAE5Qt-0CubkmHzzJ7Ng', 'profile_url'...",Baked
3,[en],397,S7lv-5qAj9bAvXxrekpHzA,https://www.yelp.com/biz/the-good-fork-brookly...,One of my favorite itineraries for showing out...,5,2018-11-21 07:52:57,"{'id': '-yEhhXT6URxh_yxHko5Gzg', 'profile_url'...",FuZ4iXW-hq5oK-U6rNm0SQ,https://www.yelp.com/biz/the-good-fork-brookly...,...,2,2019-04-04 10:10:39,"{'id': 'SwvoOnfwvtxVDZOX_JcRGA', 'profile_url'...",iTe5wvWRMrG5e3WLzgvr7A,https://www.yelp.com/biz/the-good-fork-brookly...,Was our first time to The Good Fork today - wa...,2,2019-04-21 18:57:01,"{'id': 'XKhm98gVInrRxOKPYkp9vw', 'profile_url'...",The Good Fork
4,"[fr, en, sv]",331,gomGKFtb8mZAR2XCF8dSzw,https://www.yelp.com/biz/fort-defiance-brookly...,"I came here to try ""the best Irish Coffee in t...",5,2018-12-20 18:00:52,"{'id': 'u_xu22T8i9PW7UcHPR6k9Q', 'profile_url'...",tyJfcZuhlkf7vO8M8zMQoQ,https://www.yelp.com/biz/fort-defiance-brookly...,...,4,2018-10-12 14:03:06,"{'id': 'eAw1TQ-FCrPmlgYYkG-ahg', 'profile_url'...",K5iH6pnSn5DtEGnCzReRow,https://www.yelp.com/biz/fort-defiance-brookly...,I'm going to say it- everybody who is reviewin...,5,2019-04-23 05:33:44,"{'id': 'b2RrDYWKkwBtMrGpgpoANQ', 'profile_url'...",Fort Defiance


# Next steps/to dos:

* How to process reviews/text?